In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import chardet
import numpy as np
import time

db_user = 'postgres'
db_password = '1503'
db_host = 'localhost'
db_port = '5432'
db_name = 'project'

# Создание переменной с подключением к базе данных
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

# Создание списка таблиц и пути к файлам CSV
queries = [
    {'table_name': 'ft_balance_f', 'csv_file_path': '/Users/nikitos/Desktop/import/ft_balance_f.csv'},
    {'table_name': 'ft_posting_f', 'csv_file_path': '/Users/nikitos/Desktop/import/ft_posting_f.csv'},
    {'table_name': 'md_account_d', 'csv_file_path': '/Users/nikitos/Desktop/import/md_account_d.csv'},
    {'table_name': 'md_currency_d', 'csv_file_path': '/Users/nikitos/Desktop/import/md_currency_d.csv'},
    {'table_name': 'md_exchange_rate_d', 'csv_file_path': '/Users/nikitos/Desktop/import/md_exchange_rate_d.csv'},
    {'table_name': 'md_ledger_account_s', 'csv_file_path': '/Users/nikitos/Desktop/import/md_ledger_account_s.csv'}
]

# Создание функции, которая через цикл FOR проходится поочередно по списку queries
def execute_etl_with_logging(engine, queries):
    for query in queries:
        table_name = query['table_name']
        file_path = query['csv_file_path']

        print(f'Getting started with the {table_name} table')
        
        # Запись о готовности начать работать
        ready_to_start = time.strftime('%Y-%m-%d %H:%M:%S')
    
        # Тайм-аут 5 секунд
        time.sleep(5)
    
        # Запись о начале работы с таблицей
        start_time = time.strftime('%Y-%m-%d %H:%M:%S')
        
        try:
            # Таблица 'md_currency_d' имела не стандартную кодировку, отличную от остальных таблиц,
            # поэтому определение кодировки реализовано через chardet
            if table_name == 'md_currency_d':
                with open(file_path, 'rb') as f:
                    result = chardet.detect(f.read())
                df = pd.read_csv(file_path, encoding=result['encoding'], sep=';')
            else:
                df = pd.read_csv(file_path, sep=';')
                
            # Запись количества строк в таблице на входе
            row_count_after_import = len(df)
            
            df.columns = df.columns.str.lower()

            if table_name == 'ft_balance_f':
                df['on_date'] = pd.to_datetime(df['on_date'], format='%d.%m.%Y').dt.date
            elif table_name == 'ft_posting_f':
                df['oper_date'] = pd.to_datetime(df['oper_date'], format='%d-%m-%Y').dt.date
            elif table_name == 'md_account_d':
                df['data_actual_date'] = pd.to_datetime(df['data_actual_date']).dt.date
                df['data_actual_end_date'] = pd.to_datetime(df['data_actual_end_date']).dt.date
            elif table_name == 'md_currency_d':
                df['code_iso_char'] = df['code_iso_char'].replace('˜', np.nan)
                df['data_actual_date'] = pd.to_datetime(df['data_actual_date']).dt.date
                df['data_actual_end_date'] = pd.to_datetime(df['data_actual_end_date']).dt.date
            elif table_name == 'md_exchange_rate_d':
                df.drop_duplicates(inplace=True)
                df['data_actual_date'] = pd.to_datetime(df['data_actual_date']).dt.date
                df['data_actual_end_date'] = pd.to_datetime(df['data_actual_end_date']).dt.date
            elif table_name == 'md_ledger_account_s':
                df['start_date'] = pd.to_datetime(df['start_date']).dt.date
                df['end_date'] = pd.to_datetime(df['end_date']).dt.date

            # Запрос на подключение к базе данных и отправке таблицы
            df.to_sql(table_name, engine, schema='ds', if_exists='append', index=False)
            
            # Запрос для подсчета строк в таблице после вставки их в базу данных
            with engine.connect() as conn:
                result = conn.execute(f'SELECT COUNT(*) FROM ds.{table_name}')
                row_count = result.scalar()
                
            end_time = time.strftime('%Y-%m-%d %H:%M:%S')
            print(f'Finish working with the {table_name} table, imported {row_count_after_import} and exported {row_count} rows')

            # Запись в переменную лога при завершении работы с таблицей
            log_info = {
                'ready_to_start': ready_to_start,
                'table_or_function_name': table_name,
                'start_time': start_time,
                'end_time': end_time,
                'count_record': row_count
            }
            log_to_database(engine, log_info)
            
        # на случай исключений    
        except Exception as e:
            print(f'Error importing table {table_name}: {str(e)}')
            log_info = {
                'table_or_function_name': table_name,
                'start_time': start_time,
                'end_time': time.strftime('%Y-%m-%d %H:%M:%S'),
                'count_record': 0
            }
            log_to_database(engine, log_info)
            
# Функция отправки переменных с информацией в бд логов
def log_to_database(engine, log_info):
    table_or_function_name = log_info['table_or_function_name']
    ready_to_start = log_info['ready_to_start']
    start_time = log_info['start_time']
    end_time = log_info['end_time']
    count_record = log_info['count_record']

    query = """
        INSERT INTO logs.log_info (table_or_function_name, ready_to_start, start_time, end_time, count_record)
        VALUES (%s, %s, %s, %s, %s)
    """

    try:
        with engine.connect() as conn:
            conn.execute(query, (table_or_function_name, ready_to_start, start_time, end_time, count_record))
        print('Log entry successfully added to the database.', end='\n\n')
    except Exception as e:
        print(f'Error logging to database: {str(e)}')

# Запуск функций
execute_etl_with_logging(engine, queries)

Getting started with the ft_balance_f table
Finish working with the ft_balance_f table, imported 114 and exported 114 rows
Log entry successfully added to the database.

Getting started with the ft_posting_f table
Finish working with the ft_posting_f table, imported 33892 and exported 33892 rows
Log entry successfully added to the database.

Getting started with the md_account_d table
